In [8]:
class Datagram:
    def __init__(self, source, destination):
        self.source = source
        self.destination = destination

In [9]:
class Element:
    def __init__(self, datagram, next):
        self.datagram = datagram
        self.next = next

In [ ]:
class Fila: #(desnecessaria)
    def __init__(self, size):
        self.size = size
        self.quantity = 0
        self.head = None
        self.tail = None

    def isEmpty(self):
        return self.head == None  

    def enqueue(self, datagram):
        if self.quantity < self.size:
            e = Element(datagram, None)
            if self.isEmpty():
                self.head = e
                self.tail = e
            self.tail.next = e
            self.tail = e
            self.quantity += 1
            return True
        return False

    def dequeue(self):
        if not self.isEmpty():
            datagram = self.head.datagram
            self.head = self.head.next
            self.quantity -= 1
            return datagram
        return None

In [10]:
class Queue:
    def __init__(self, size):
        self.queue = []
        self.size = size

    def is_empty(self):
        return len(self.queue) == 0
    
    def enqueue(self, datagram):
        if len(self.queue) < self.size:
            self.queue.append(datagram)
            return True
        return False

    def dequeue(self):
        if not self.isEmpty():
            return self.queue.pop(0)
        return None

In [11]:
class Event:
    def __init__(self, instant, destination, datagram):
        self.instant = instant
        self.destination = destination
        self.datagram = datagram
    

In [12]:
class Router:
    def __init__(self, address, size):
        self.address = address
        self.adjacents = {}
        self.datagrams = []
        self.size = size

    def link(self, adjacent, delay):
        if adjacent not in self.adjacents:
            queue = Queue(self.size)
            self.adjacents[adjacent] = (queue, delay)

    def receive(self, datagram):
        self.datagrams.append(datagram)

    def process(self, instant):
        events = []
        for adjacent in self.adjacents:
            datagram = self.adjacents[adjacent][0].dequeue()
            if datagram != None:
                event = Event(instant + self.adjacents[adjacent][1], adjacent, datagram)
                events.append(event)
        return events
    
    def routing(self, action):
        """mandar os datagramas em self.datagramas para uma determinada fila"""

In [13]:
class Network:
    def __init__(self, size):
        self.size = size
        self.quantity = 0
        self.routers = {}

    def add(self, router: Router):
        if self.quantity < self.size:
            if router.address not in self.routers:
                self.routers[router.address] = router
                self.quantity += 1
                return True
        return False

In [14]:
class Scheduler:
    def __init__(self, inicial_instant, network, size):
        self.instant = inicial_instant
        self.network = network
        self.size = size
        self.events = []
    
    def schedule(self, instant, router, datagram):
        if len(self.events) == self.size:
            return False
        self.events.append(Event(instant, router, datagram)) #aqui não deveria ser: Event(instant, datagram.source, datagram)?
        return True
    
    def process(self, action):
        for event in self.events:
            if event.instant == self.instant:
                event.router.receive(event.datagram)
        self.events = [event for event in self.events if event.instant != self.instant]
        for router in self.network.routers:
            router.routing(action[i]) ### algo assim?
            event = router.process(self.instant)
            if event is not None:
                self.events.append(event)
        self.instant += 1

obs: Se não for ter criação de evento AGENDADO, o método schedule() parece pouco útil, podemos inserir um novo datagrama diretamente em datagrams do router de origem 